<a href="https://colab.research.google.com/github/aetev/Hearth-Stone-Python-Simulator/blob/main/card_enc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import json

# Assuming AtomicCards.json is in your Google Drive, adjust the path if necessary
file_path = '/content/drive/MyDrive/AtomicCards.json'


with open(file_path, 'r') as f:
  data = json.load(f)['data'])

In [11]:
def filter_legal_cards(data):
    legal_commander_data = {}
    for card_name, card_data in data.items():
        try:
            if card_data[0]['legalities']['commander'] == 'Legal':
                legal_commander_data[card_name] = card_data[0] # Store only the first element of card_data
        except (KeyError, IndexError):
            # Handle cases where 'legalities' or 'commander' key is missing or card_data is empty
            pass  # Or print a message, log it, etc.
    return legal_commander_data

filtered_data = filter_legal_cards(data)


In [23]:
first_key = next(iter(filtered_data))
first_key = next(iter(filtered_data))
first_key = next(iter(filtered_data))
first_value = filtered_data[first_key]

print("Key:", first_key)
print("text:", first_value['text'])
print("keywords:", first_value['keywords'])

Key: "Lifetime" Pass Holder
text: This creature enters tapped.
When this creature dies, open an Attraction.
Whenever you roll to visit your Attractions, if you roll a 6, you may return this card from your graveyard to the battlefield.
keywords: ['Open an Attraction', 'Roll to Visit Your Attractions']
